In [ ]:
# Import the libraries
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from google.colab import drive
import urllib.request  # Add this import
from google.colab.patches import cv2_imshow

# Mount Google Drive to access files
drive.mount('/content/drive')

# Replace the video_url with the sharable link of video file in Google Drive
video_url = 'https://drive.google.com/uc?id=1nOI6pIpUJ38lLSaz7Jv60exiwtvaifGW'

# Download the video from the URL and save it as 'video.mp4' in the current directory
video_file = "video.mp4"
urllib.request.urlretrieve(video_url, video_file)

# Open the video file for reading
video_capture = cv2.VideoCapture(video_file)

# Load the YOLO model
model_url = "https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2"  # Use a different version of the YOLO model
yolo_model = hub.load(model_url)

# Alternative implementation of CentroidTracker
class CentroidTracker:
    def __init__(self):
        self.next_object_id = 0
        self.objects = {}

    def update(self, input_centroids):
        new_objects = {}

        for centroid in input_centroids:
            object_id = self.next_object_id
            self.next_object_id += 1
            new_objects[object_id] = centroid

        self.objects = new_objects
        return self.objects

# Initialize the CentroidTracker
centroid_tracker = CentroidTracker()

while True:
    ret, frame = video_capture.read()
    if not ret:
        break

    # Convert the frame to RGB format
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Perform object detection using YOLO
    input_image = tf.image.convert_image_dtype(frame_rgb, dtype=tf.uint8)  # Convert frame to tf.uint8
    input_image = tf.expand_dims(input_image, axis=0)  # Add batch dimension
    outputs = yolo_model(input_image)
    boxes = outputs["detection_boxes"]
    scores = outputs["detection_scores"]

    # Extract centroids from the detected boxes
    input_centroids = []
    for box, score in zip(boxes[0], scores[0]):
        if score > 0.3:  # Consider boxes with different scores
            ymin, xmin, ymax, xmax = box
            x_center = (xmin + xmax) / 2
            y_center = (ymin + ymax) / 2
            centroid = (int(x_center * frame.shape[1]), int(y_center * frame.shape[0]))
            input_centroids.append(centroid)

            # Draw bounding boxes
            x, y, w, h = int(xmin * frame.shape[1]), int(ymin * frame.shape[0]), int((xmax - xmin) * frame.shape[1]), int((ymax - ymin) * frame.shape[0])
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Use the CentroidTracker to track objects
    tracked_objects = centroid_tracker.update(input_centroids)

    for object_id, centroid in tracked_objects.items():
        x, y = centroid  # Unpack the x and y coordinates from the centroid tuple
        cv2.circle(frame, (x, y), 4, (0, 255, 0), -1)
        cv2.putText(frame, f"ID {object_id}", (x - 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2_imshow(frame)  # Use cv2_imshow to display the frame

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()